# 1. Introduction
I am going to present a set of text and numeric feature extraction techniques. I start with importing modules and loading the files.

In [1]:
import pylab as pl # linear algebra + plots
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import gc
import xgboost as xgb
import lightgbm as lgb
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict, Counter
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob
from scipy.stats import pearsonr
from scipy.sparse import hstack
from multiprocessing import Pool


train = pd.read_csv('train-3.csv')
test = pd.read_csv('test-3.csv', low_memory=False)
resources = pd.read_csv('resources.csv')

# 2. Data Cleaning
As explained in the data description page, the essay column formats had changed on 2016-05-17, and thereafter, there are only 2 essays; essay 1 matches to the combination of essays 1&2 and new essay 2 is  equal to old essays 3&4.
I am going to move the contents of 'project_essay_2' to 'project_essay_4' when essay 4 is nan. In the next step I will combine 1&2 and 3&4 to creaate a unifrom dataset.



In [2]:
target = 'project_is_approved'
#create two columns to distinquish train and test data
train['tr'] = 1; test['tr'] = 0
train['ts'] = 0; test['ts'] = 1

df= pd.concat((train,test))

# Moving essay 2 to essay 4
df.loc[df.project_essay_4.isnull(), ['project_essay_4','project_essay_2']] = df.loc[df.project_essay_4.isnull(), ['project_essay_2','project_essay_4']].values
# Filling NaNs with empty string
df[['project_essay_2','project_essay_3']] = df[['project_essay_2','project_essay_3']].fillna('')

# Joining essay 1 and essay 2
df['project_essay_1'] = df.apply(lambda row: ' '.join([str(row['project_essay_1']), 
                                                     str(row['project_essay_2'])]), axis=1)
# Joining essay 3 and essay 4
df['project_essay_2'] = df.apply(lambda row: ' '.join([str(row['project_essay_3']),
                                                     str(row['project_essay_4'])]), axis=1)
# Droping essay 3 and essay 4 from the overall dataframe
df= df.drop(['project_essay_3', 'project_essay_4'], axis=1)



# 3. Resource Features

For each application, some resources are asked in the resource file. First of all, I am going to extract how many items and at what prices are requested. Minimum, maximum and average price and quantity of each item and for all requested items per application can be important in the decision-making process.

Also, I am going to combine the resource description columns and make a new text column in table df.

In [3]:
# Adding priceAll by multiplying quantity and price
resources['priceAll'] = resources['quantity']*resources['price']

#group by id column in resources table and perform aggregation functions on resources columns
temp = resources.groupby('id').agg({'description':'count',
                            'quantity':'sum',
                            'price':'sum',
                            'priceAll':'sum'}).rename(columns={'description':'items'})
temp['avgPrice'] = temp.priceAll / temp.quantity
numFeatures = ['items', 'quantity', 'price', 'priceAll', 'avgPrice']

# Adding three more functions to the new dataframe (temp)
for func in ['min', 'max', 'mean']:
    temp = temp.join(resources.groupby('id').agg({'quantity':func,
                                          'price':func,
                                          'priceAll':func}).rename(
                                columns={'quantity':func+'Quantity',
                                         'price':func+'Price',
                                         'priceAll':func+'PriceAll'}).fillna(0))
    numFeatures += [func+'Quantity', func+'Price', func+'PriceAll']

#group by on id to join description of resources foe each id    
temp = temp.join(resources.groupby('id').agg(
    {'description':lambda x:' '.join(x.values.astype(str))}).rename(
    columns={'description':'resource_description'}))

df = df.join(temp, on='id')

# In the donorschoose website, the price is categorized by these bins:
df['price_category'] = pl.digitize(df.priceAll, [0, 50, 100, 250, 500, 1000, pl.inf])
numFeatures.append('price_category')

# Add three more columns
for c in ['Quantity', 'Price', 'PriceAll']:
    df['max%s_min%s'%(c,c)] = df['max%s'%c] - df['min%s'%c]
    numFeatures.append('max%s_min%s'%(c,c))


In [4]:
le = LabelEncoder()
df['teacher_id'] = le.fit_transform(df['teacher_id'])
df['teacher_gender_unknown'] = df.teacher_prefix.apply(lambda x:int(x not in ['Ms.', 'Mrs.', 'Mr.']))
numFeatures += ['teacher_number_of_previously_posted_projects','teacher_id','teacher_gender_unknown']

statFeatures = []
for col in ['school_state', 'teacher_id', 'teacher_prefix', 'teacher_gender_unknown', 'project_grade_category', 'project_subject_categories', 'project_subject_subcategories', 'teacher_number_of_previously_posted_projects']:
    Stat = df[['id', col]].groupby(col).agg('count').rename(columns={'id':col+'_stat'})
    Stat /= Stat.sum()
    df = df.join(Stat, on=col)
    statFeatures.append(col+'_stat')

# 4. Sentimental Analysis

In [5]:
dateCol = 'project_submitted_datetime'
def getTimeFeatures(T):
    T['year'] = T[dateCol].apply(lambda x: x.year)
    T['month'] = T[dateCol].apply(lambda x: x.month)
    T['day'] = T[dateCol].apply(lambda x: x.day)
    T['dow'] = T[dateCol].apply(lambda x: x.dayofweek)
    T['hour'] = T[dateCol].apply(lambda x: x.hour)
    T['days'] = (T[dateCol]-T[dateCol].min()).apply(lambda x: x.days)
    return T

df[dateCol] = pd.to_datetime(df[dateCol])
df = getTimeFeatures(df)

P_tar = df[df.tr==1][target].mean()
timeFeatures = ['year', 'month', 'day', 'dow', 'hour', 'days']
for col in timeFeatures:
    Stat = df[['id', col]].groupby(col).agg('count').rename(columns={'id':col+'_stat'})
    Stat /= Stat.sum()
    df = df.join(Stat, on=col)
    statFeatures.append(col+'_stat')

numFeatures += timeFeatures
numFeatures += statFeatures

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260115 entries, 0 to 78034
Data columns (total 56 columns):
id                                                   260115 non-null object
project_essay_1                                      260115 non-null object
project_essay_2                                      260115 non-null object
project_grade_category                               260115 non-null object
project_is_approved                                  182080 non-null float64
project_resource_summary                             260115 non-null object
project_subject_categories                           260115 non-null object
project_subject_subcategories                        260115 non-null object
project_submitted_datetime                           260115 non-null datetime64[ns]
project_title                                        260115 non-null object
school_state                                         260115 non-null object
teacher_id                                   

In [8]:
(df[dateCol]-df[dateCol].min()).apply(lambda x: x.days)

0        205
1        364
2        249
3        107
4        101
5        192
6        126
7         98
8        139
9        150
10       316
11       131
12       225
13       202
14       281
15       205
16       130
17       273
18         0
19        24
20       114
21       156
22        81
23        50
24       178
25       123
26       261
27       167
28       187
29       181
        ... 
78005    343
78006    361
78007    344
78008    367
78009    348
78010    349
78011    350
78012    353
78013    358
78014    341
78015    341
78016    364
78017    344
78018    355
78019    354
78020    345
78021    347
78022    365
78023    365
78024    356
78025    356
78026    358
78027    364
78028    349
78029    342
78030    345
78031    363
78032    355
78033    357
78034    365
Name: project_submitted_datetime, Length: 260115, dtype: int64